In [1]:
%pip install openai pydantic jinja2 python_dotenv ipywidgets d20

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import re
import tomllib
import functools

from jinja2 import Template
from pydantic import BaseModel
from openai import OpenAI, AzureOpenAI
from dotenv import load_dotenv
from d20 import roll

import ipywidgets as widgets
from IPython.display import display, clear_output

from typing import Literal, Union

load_dotenv()

with open("data/example_the_haunting.toml", "rb") as f:
    config = tomllib.load(f)

message_pattern = re.compile(r"^([\w_ *]+)\:[ \s]*(.+)*$")
class Message(BaseModel):
    # Message from chatbot
    role: Literal["system", "assistant", "user"]
    content: str   

    # parse the content into dictionary
    def dict(self) -> dict:
        current_key = None
        parsed = {}
        lines = self.content.splitlines()
        # parse the content line by line
        for line in lines:
            res = message_pattern.search(line)
            if res is not None:
                current_key = res.group(1)
                if res.group(2) is not None:
                    parsed[current_key] = res.group(2).strip()
                else:
                    parsed[current_key] = ""
            elif current_key is not None and line.strip() != "":
                parsed[current_key] += "\n" + line.strip()
        return parsed

class Chapter(BaseModel):
    id: str
    next: str|None = None
    max_turns: int
    backgrounds: list[str]
    rules: list[str]
    initial: str
    
    def __init__(self, **kargs):
        super().__init__(**kargs)
    
    def get_messages(self) -> list[Message]:
        rendered_rules = render_rules(self.rules)
        system_content = Template(system).render({"rules": rendered_rules, "backgrounds":self.backgrounds, "character": character})
        initial_content = Template(self.initial).render({"rules": self.rules, "character": character})
        return [
            Message(role="system", content=system_content),
            Message(role="assistant", content=initial_content)
        ]

version = config["version"] # version of the config
system = config["system"] # system prompt template message
character = config["character"] # character information
chapters = config["chapters"] # chapters list
item_list = config["list"]["items"] # items list
enemy_list = config["list"]["enemies"] # items list

messages:list[Message] =[] # chat messages

tags = [] # tags list
items = character["items"] # items list

print(f"Version: {version}, Chapters: {len(chapters)}")

# find the chapter by id
def getChapterById(id:str) -> Chapter:
    for r in chapters:
        if r["id"] == id:
            return Chapter(**r) 
    return None

# next chapter find and initialize
def choose_next_chapter():
    global current_chapter, messages
    if current_chapter.next is not None:
        next = Template(current_chapter.next).render({"items": items, "tags": tags})
        print("Next round: ", next)
        next_chapter = getChapterById(next)
        if next_chapter is not None:
            current_chapter = next_chapter
            initials = current_chapter.get_messages()
            print(initials[1].content)
            show_controls(initials[1])
            messages = []
        else:
            print(f"Chapter '{next}' not found.")
    else:
        print("No next chapter found.")


# cout the messages by role, used for chapter's max_turns check
def count_messages(messages:list[Message], role:str) -> int:
    return len([msg for msg in messages if msg.role == role])

# chat with Azure OpenAI
def chat_with_azure(messages:list[Message], on_message: Union[callable, None] = None) -> Message:
    client = AzureOpenAI(api_key=os.environ.get("AZURE_OPENAI_API_KEY"), 
                         azure_endpoint=os.environ.get("AZURE_OPENAI_ENDPOINT"),
                         api_version=os.environ.get("AZURE_OPENAI_API_VERSION")
                         )
    stream = client.chat.completions.create(
        model=os.environ.get("AZURE_OPENAI_MODEL_NAME"),
        messages=messages,
        stream=True,
        max_tokens=4000,
        temperature=float(os.environ.get("OPENAI_TEMPERATURE"))
    )

    generated = ""
    for chunk in stream:
        if len(chunk.choices) > 0 and chunk.choices[0].delta.content is not None:
            delta = chunk.choices[0].delta.content
            if (on_message is not None):
                on_message(chunk.choices[0].delta.content)
            generated += delta
    return Message(role="assistant", content=generated)
    
# chat with OpenAI
def chat(messages:list[Message], on_message: Union[callable, None] = None) -> Message:
    client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
    stream = client.chat.completions.create(
        model=os.environ.get("OPENAI_MODEL_NAME"),
        messages=messages,
        stream=True,
        max_tokens=4000,
        temperature=float(os.environ.get("OPENAI_TEMPERATURE"))
    )

    generated = ""
    for chunk in stream:
        if chunk.choices[0].delta.content is not None:
            delta = chunk.choices[0].delta.content
            if (on_message is not None):
                on_message(chunk.choices[0].delta.content)
            generated += delta
    return Message(role="assistant", content=generated)

# render each rule from the chapter
def render_rules(rules:list[str]) -> str:
    lists = []
    for rule in rules:
        res = Template(rule).render({"character": character, "tags":tags, "items": items})
        if res is not None and res != "":
            lists.append(res)
    return lists

# do chat with the chatbot, and show the message with control buttons...
def do_chat(content:str|None, on_message: Union[callable, None] = None):
    if content is not None:
        turn = count_messages(messages, 'user') + 1
        if turn >= current_chapter.max_turns:
            content += f"\nwhisper: MAX_TURNS is reached."
        messages.append(Message(role="user", content=content))
        print(content)

    # change it to chat_with_azure to use Azure OpenAI
    concat = current_chapter.get_messages() + messages
    # for msg in concat:
    #     print(f"{msg.role}: {msg.content}")
    # print("----------------------------------------------")

    msg = chat_with_azure(concat, on_message)

    # print all the messages:
    show_controls(msg)
    messages.append(msg)
        
# handle player's action
def on_action(b, index:int):
    user_msg = f"I select '{index}'."
    do_chat(user_msg, lambda x: print(x, end=""))

# handle player's skill check
def on_skill(b, skill:str, difficulty:str):
    roll_num = roll("1d100").total
    if roll_num < 5:
        user_msg = f"i roll the dice for '{roll_num}' and get a CRITICAL SUCCESS!"
        do_chat(user_msg, lambda x: print(x, end=""))
        return
    if roll_num >= 95:
        user_msg = f"i roll the dice for '{roll_num}' and get a CRITICAL FAILURE!"
        do_chat(user_msg, lambda x: print(x, end=""))
        return

    difficulty_num = 15 if skill not in character["skills"] else character["skills"][skill]
    if difficulty == "hard":
        difficulty_num = int(difficulty_num * 0.5)
    if difficulty == "extreme":
        difficulty_num = int(difficulty_num * 0.2)
    
    print(f"rolled {roll_num} against {difficulty_num} on skill '{skill}'...")
    
    if roll_num <= difficulty_num:
        user_msg = f"i roll the dice for '{skill.upper()}' and get a success!"
    else:
        user_msg = f"i roll the dice for '{skill.upper()}' and get a failure."

    do_chat(user_msg, lambda x: print(x, end=""))

# handle continue
def on_continue(b):
    user_msg = f"Continue..."
    do_chat(user_msg, lambda x: print(x, end=""))

# handle chapter's ending, and show next chapter
def on_next(b):
    choose_next_chapter()
        
# handle player's retry, it will remove the last assistant message,
# and re-ask the chatbot
def on_retry(b):
    # delete last assistant message
    messages.pop()
    do_chat(None, lambda x: print(x, end=""))

# handle player's item usage
def on_item(b, item:dict):
    user_msg = Template(item["description"]).render({"character": character, "tags":tags, "items": items})
    user_msg = f"I use '{item['id']}' - {user_msg}"
    do_chat(user_msg, lambda x: print(x, end=""))

# handle player's attack
def on_attack(b, target:dict, weapon:dict):
    requirement = weapon["requirement"]
    skill_check_result = roll("1d100").total
    print("skill check result:", skill_check_result)
    if requirement:
        # 遍历requirement的技能进行检定
        for skill in requirement:
            skill_value = character["skills"]["requirement"] or 20
            
            # 执行技能检定
            if skill_check_result > skill_value:
                user_msg = f"I tried to use '{weapon['id']}', but failed."
                do_chat(user_msg, lambda x: print(x, end=""))
                return
            else:
                # player attack target
                hit = roll(weapon['stats']['attack']).total
                target["stats"]["hp"] -= hit
                print("player's attack:", hit)
                if target["stats"]["hp"] <= 0:
                    user_msg = f"I hit {target['name']}, and {target['name']} is dead..."
                    do_chat(user_msg, lambda x: print(x, end=""))
                    return

                user_msg = f"I hit {target['name']}, and {target['name']} lost some hp..."

                # target attack player
                hit = roll("1d6").total
                character["stats"]["hp"] -= hit
                print("target's attack:", hit)
                if character["stats"]["hp"] <= 0:
                    user_msg = f"I got hit by {target['name']}, and player is dead...\nWHISPER: GAME_OVER"
                    do_chat(user_msg, lambda x: print(x, end=""))
                    return

                user_msg += f"\nand I got hit by {target['name']}, lost some hp..."
                do_chat(user_msg, lambda x: print(x, end=""))

action_pattern = re.compile(r"(\d+). *(.+)")
# the main function to show the controls
def show_controls(msg:Message):
    dict = msg.dict()
    buttons = []

    if "choices" in dict.keys():
        # handle player's choices
        matches = action_pattern.finditer(dict["choices"])
        for match in matches:
            btn = widgets.Button(description=match.group(0), layout=widgets.Layout(width="fit-content"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill_check" in dict.keys() and "difficulty" in dict.keys():
        # handle skill check
        btn = widgets.Button(description=f"{dict['skill_check']} - {dict['difficulty']}", layout=widgets.Layout(width="fit-content"))
        btn.on_click(functools.partial(on_skill, skill=dict['skill_check'], difficulty=dict['difficulty']))
        buttons.append(btn)
    elif "battle_with" in dict.keys():
        enemy = next((x for x in enemy_list if x['id'] == dict["battle_with"]), None)
        # print("start battle with:", enemy.name)
        btn = widgets.Button(description=f"Attack", layout=widgets.Layout(width="fit-content"))
        btn.on_click(functools.partial(on_attack, target=enemy))
        buttons.append(btn)
        for item_id in items:
            item = next((x for x in item_list if x['id'] == item_id), None)
            if item["item_type"] == "weapon":
                btn = widgets.Button(description=f"{item["id"]}", layout=widgets.Layout(width="fit-content"))
                btn.on_click(functools.partial(on_attack, target=enemy, weapon=item))
                buttons.append(btn)
        pass
    elif "ending" in dict.keys():
        # handle chapter ending, show next chapter button
        btn = widgets.Button(description=f"NEXT", layout=widgets.Layout(width="fit-content"))
        btn.on_click(on_next)
        buttons.append(btn)
    elif "game_over" in dict.keys():
        # handle game over
        print("\nGame Over - Hit 'Run All' button to restart again.")
    else:
        # If there is no choices or endings, show continue button
        btn = widgets.Button(description="Continue", layout=widgets.Layout(width="fit-content"))
        btn.on_click(on_continue)
        buttons.append(btn)
    
    # add tag from generated message
    if "add_tag" in dict.keys() and dict["add_tag"] not in tags:
        tags.append(dict["add_tag"])

    # remove tag from generated message
    if "remove_tag" in dict.keys() and dict["remove_tag"] in tags:
        tags.remove(dict["remove_tag"])
    
    # add item from generated message
    if "add_item" in dict.keys() and dict["add_item"] not in item_list:
        item = dict["add_item"]
        found = next((x for x in item_list if x['id'] == item), None)
        if (found is None):
            print(f"\nItem '{dict['add_item']}' not found.")
        else:
            items.append(item)
    
    # remove item from generated message
    if "remove_item" in dict.keys() and dict["remove_item"] in items:
        items.remove(dict["remove_item"])
    
    controls = []

    # retry buttons
    if len(messages) > 0:
        retry_btn = widgets.Button(description="Retry", layout=widgets.Layout(width="fit-content"))
        retry_btn.on_click(on_retry)
        controls.append(retry_btn)

    # show tag list
    if len(tags) > 0:
        tags_lbl = widgets.Label(value=f"Tags: {', '.join(tags)}")
        controls.append(tags_lbl)

    # inventory buttons
    inventory = [widgets.Label(value="Inventory:")]
    if len(items) > 0:
        for item in items:
            found = next((x for x in item_list if x['id'] == item), None)
            if found:
                item_btn = widgets.Button(description=found["id"], layout=widgets.Layout(width="fit-content"))
                item_btn.on_click(functools.partial(on_item, item=found))
                inventory.append(item_btn)
            else:
                print(f"\nItem '{item}' not found in the list.")
        controls.append(widgets.HBox(inventory))
    
    display(widgets.VBox(buttons))

    if (len(controls) > 0):
        display(widgets.HBox(controls))

current_chapter = getChapterById("r2-0")

Version: 0.0.3, Chapters: 4


In [3]:
def attack_with(skill_id, difficulty):
    roll_num = roll("1d100").total

    print("roll num:", roll_num)

    if (roll_num < 5): return "critical_success"
    if (roll_num >= 95): return "critical_failure"

    skill_value = 0
    if skill_id in character["skills"]:
        skill_value = character["skills"][skill_id]
        if difficulty == "hard":
            skill_value = int(skill_value * 0.5)
        if difficulty == "extreme":
            skill_value = int(skill_value * 0.2)
    else:
        skill_value = 15

    print("skill value:", skill_value)
    if roll_num <= skill_value:
        return "success"

    return "failure"

print(attack_with("shotgun", "hard"))


roll num: 72
skill value: 15
failure


In [ ]:
initials = current_chapter.get_messages()
print(initials[1].content)
show_controls(initials[1])

background: front_door
narration: 
You're standing in front of the house. The brick house stands wedged between two newer, taller office buildings, swallowed by their looming shadows. The modern facades starkly contrast the old house, making it seem like a relic from a forgotten time. Its weathered, cracked bricks are covered in grime, dulling their once vibrant color.

In spite of your experience, you still got shivered by what you have learnt about.

choices:
1. Go inside.
2. Peak through the window.
3. Walk around the house.


I select '1'.
narration: 
As you push the heavy, wooden front door open, a gust of stale air rushes past you, carrying with it the scent of mildew and decay. No sooner than you step inside, a large black dog charges at you from the shadows, its eyes wild and teeth bared.

skill_check: dodge
difficulty: normal

rolled 10 against 80 on skill 'dodge'...
i roll the dice for 'DODGE' and get a success!
narration: 
With a swift sidestep, you dodge the lunging dog. It barrels past you, growling in frustration before scampering away into the deeper shadows of the house. You're now in the dimly lit hallway of the first floor. The air is thick with dust, and the floorboards creak under your weight.

The first floor consists of a living room, a study, and a bathroom.

choices:
1. Explore the living room.
2. Check out the study.
3. Head to the bathroom.
4. Listen carefully for any unusual sounds.

I select '1'.
narration: 
You step into the living room, a space that speaks of old grandeur now faded. Heavy curtains block out most of the light, and furniture draped in white sheets gives the room a ghostly ambiance. As you move further inside, your foot nudges something on the floor.

Lifting the corner of a dusty sheet, you find a framed photograph lying face-down. You pick it up and wipe off the dust, revealing a family portrait of the Macarios. They smile back at you, their happiness a stark contrast to the grim atmosphere of the house.

add_item: macario_family_photo

choices:
1. Continue exploring the living room.
2. Check out the study.
3. Head to the bathroom.
4. Listen carefully for any unusual sounds.
Item 'macario_family_photo' not found.


I select '2'.
narration: 
You enter the study, a room cluttered with books, papers, and religious icons. Crucifixes and statues of Madonnas are scattered everywhere, some standing proudly on shelves, others lying forgotten on the floor.

Amidst the chaos, a large, antique desk catches your eye, papers strewn across its surface.

skill_check: spot_hidden
difficulty: normal

rolled 88 against 50 on skill 'spot_hidden'...
i roll the dice for 'SPOT_HIDDEN' and get a failure.
narration: 
Despite your best efforts, the clutter proves too much, and you find nothing of immediate importance among the scattered papers and religious artifacts. The room feels heavy with a sense of desperation, as if someone was seeking protection or answers they couldn't find.

choices:
1. Try searching the desk again.
2. Head to the bathroom.
3. Return to the living room.
4. Listen carefully for any unusual sounds.

I select '4'.
narration: 
You stand still, holding your breath as you strain your ears. The house seems to groan under its own weight, the silence punctuated by the distant, eerie creaking of wood. Then, distinctly, a soft creak echoes through the house—it seems to come from the second floor.

ending: You realize the noise is not just the house settling. Someone, or something, is up there.